In [1]:
import argparse
import h5py
import numpy as np
import os
import pickle
import shutil
import yaml
import json
import click
from IPython.display import Image, display
from pprint import pprint
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn

import vqa.lib.engine as engine
import vqa.lib.utils as utils
import vqa.lib.logger as logger
import vqa.lib.criterions as criterions
import vqa.datasets as datasets
import vqa.models as models

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open('options/vqa2/mutan_noatt_train_cex.yaml', 'r') as handle:
    options = yaml.load(handle)
options['vgenome'] = None

In [3]:
options

{'coco': {'arch': 'fbresnet152',
  'dir': 'data/coco',
  'mode': 'noatt',
  'size': 448},
 'logs': {'dir_logs': 'logs/vqa2/mutan_noatt_train'},
 'model': {'arch': 'MutanNoAtt',
  'classif': {'dropout': 0.5},
  'fusion': {'R': 10,
   'activation_q': 'tanh',
   'activation_v': 'tanh',
   'dim_hq': 360,
   'dim_hv': 360,
   'dim_mm': 360,
   'dim_q': 2400,
   'dim_v': 2048,
   'dropout_hq': 0,
   'dropout_hv': 0,
   'dropout_q': 0.5,
   'dropout_v': 0.5},
  'seq2vec': {'arch': 'skipthoughts',
   'dir_st': 'data/skip-thoughts',
   'dropout': 0.25,
   'fixed_emb': False,
   'type': 'BayesianUniSkip'}},
 'optim': {'batch_size': 64, 'epochs': 100, 'lr': 0.0001},
 'vgenome': None,
 'vqa': {'dataset': 'VQA2',
  'dir': 'data/vqa2',
  'maxlength': 26,
  'minwcount': 0,
  'nans': 2000,
  'nlp': 'mcb',
  'pad': 'right',
  'samplingans': True,
  'trainsplit': 'train'}}

In [4]:
trainset = datasets.factory_VQA(options['vqa']['trainsplit'],
                                options['vqa'],
                                options['coco'],
                                options['vgenome'])

In [ ]:
trainset

In [ ]:
trainset.dataset_img

In [ ]:
trainset[0].keys()

In [5]:
train_loader = trainset.data_loader(batch_size=options['optim']['batch_size'],
                                    num_workers=1,
                                    shuffle=True)

In [ ]:
# sample = next(iter(train_loader))

In [ ]:
trainset.dataset_img.index_to_name

In [ ]:
trainset.dataset_img.name_to_index

In [ ]:
train_loader.dataset

In [ ]:
for i, sample in enumerate(train_loader):
    print(sample.keys())
    break

In [ ]:
sample["visual"]

In [ ]:
sample['question_id']

In [6]:
train_examples_list = pickle.load(open('data/vqa2/processed/nans,2000_maxlength,26_minwcount,0_nlp,mcb_pad,right_trainsplit,train/trainset.pickle', 'rb'))
qid_to_example = {ex['question_id']: ex for ex in train_examples_list}

In [ ]:
len(train_examples_list)

In [ ]:
trainset.dataset_img.name_to_index['COCO_train2014_000000458752.jpg']

In [7]:
comp_pairs = json.load(open("data/vqa2/raw/annotations/v2_mscoco_train2014_complementary_pairs.json", "r"))

comp_q = {}
for q1, q2 in comp_pairs:
    comp_q[q1] = q2
    comp_q[q2] = q1
    
len(comp_q)

395850

In [8]:
qid = sample['question_id'][1]

ex1 = qid_to_example[qid]
ex2 = qid_to_example[comp_q[qid]]

display(Image(filename=os.path.join('data/coco/raw/train2014', ex1['image_name'])))
print("Q1: {}\nA1: {}\n".format(ex1['question'], ex1['answer']))

display(Image(filename=os.path.join('data/coco/raw/train2014', ex2['image_name'])))
print("Q2: {}\nA2: {}\n".format(ex2['question'], ex2['answer']))

NameError: name 'sample' is not defined

In [ ]:
sample['question_id'][0]

In [ ]:
ex1

In [14]:
for i, sample in enumerate(train_loader):
    break

In [ ]:
qid_to_example[sample['question_id'][0]]

In [9]:
knns = np.load("data/coco/extract/arch,fbresnet152_size,448/knn/knn_results_trainset.npy").reshape(1)[0]
knn_idx = knns["indices"]

In [10]:
knn_idx.shape

(82783, 25)

In [15]:
# Get KNNs for original images
iids_orig = [trainset.dataset_img.name_to_index[image_name] for image_name in sample['image_name']]

# TODO
knns_batch = [list(knn_idx[i]) for i in iids_orig]

In [16]:
f = h5py.File('data/coco/extract/arch,fbresnet152_size,448/trainset.hdf5', 'r')
features = f.get('noatt')
features[0].shape

(2048,)

In [17]:
knn_features = [[features[i] for i in knns_batch[j]] for j in tqdm(range(len(knns_batch)))]

100%|██████████| 64/64 [00:00<00:00, 214.97it/s]


In [21]:
# Get complementary q_ids (or None if question has no compliment)
qids_comp = [comp_q[qid] if qid in comp_q else None for qid in sample['question_id']]

In [23]:
no_compliment = 0
missing_example = 0

image_names_comp = []

for qid in qids_comp:
    if not qid:
        no_compliment += 1
        image_names_comp.append(None)
        continue
    if qid not in qid_to_example:
        missing_example += 1
        image_names_comp.append(None)
        continue
    
    image_names_comp.append(qid_to_example[qid]['image_name'])
    
print(len(image_names_comp))
print(no_compliment, missing_example)

64
10 3


In [24]:
# THE COUNTEREXAMPLE IS FREQUENTLY NOT CONTAINED IN KNNS

comp_in_knns = comp_out_knns = 0

for i, name in enumerate(image_names_comp):
    if name:
        iid = trainset.dataset_img.name_to_index[name]
        if iid in knns_batch[i]:
            comp_in_knns += 1
        else:
            comp_out_knns += 1
                
print(comp_in_knns, comp_out_knns)

5 46


In [ ]:
i = 1

qid = sample['question_id'][i]

ex1 = qid_to_example[qid]
ex2 = qid_to_example[comp_q[qid]]

knn_names = [trainset.dataset_img.index_to_name[nn] for nn in knns_batch[i]]

print(ex2['image_name'] in knn_names)

display(Image(filename=os.path.join('data/coco/raw/train2014', ex1['image_name'])))
print("Q1: {}\nA1: {}\n".format(ex1['question'], ex1['answer']))

display(Image(filename=os.path.join('data/coco/raw/train2014', ex2['image_name'])))
print("Q2: {}\nA2: {}\n".format(ex2['question'], ex2['answer']))

for name in knn_names:
    display(Image(filename=os.path.join('data/coco/raw/train2014', name)))

In [ ]:
test = [ex["question_id"] for ex in train_examples_list]
print(len(test), len(set(test)))

In [ ]:
raw_questions = json.load(open("data/vqa2/raw/annotations/OpenEnded_mscoco_train2014_questions.json", "r"))


In [ ]:
len(raw_questions['questions'])

In [ ]:
lens = np.array([len(q['question'].split()) for q in raw_questions['questions']])

In [ ]:
np.argmax(lens)

In [ ]:
raw_questions['questions'][207064]

In [25]:
iids_comp = [trainset.dataset_img.name_to_index[name] if name is not None else None for name in image_names_comp]

good_example_idxs = []
comp_idxs = []
for i, iid in enumerate(iids_comp):
    if iid is not None:
        if iid in knns_batch[i]:
            good_example_idxs.append(i)
            comp_idxs.append(knns_batch[i].index(iid))

In [26]:
knn_features = [np.array([features[i] for i in knns_batch[j]]) for j in good_example_idxs]

In [27]:
comp_idxs

[1, 7, 10, 3, 9]

In [ ]:
sample.keys()

In [41]:
orig = {
    # Index of original image within knns is always zero
    'idxs': np.zeros(len(good_examples)),
    'q': sample['question'][good_example_idxs],
    'q_id': sample['question_id'][good_example_idxs],
    'a': sample['question'][good_example_idxs],
}
comp = {
    # Index (0 - 24) of comp within knns of original image
    'idxs': comp_idxs,
    'q_id': [qids_comp[q_id] for q_id in good_example_idxs]
}
neighbors = {
    'v': knn_features,
}

good_examples = {
    'orig': orig,
    'comp': comp,
    'neighbors': neighbors,
}

In [38]:
a = good_examples['neighbors']['v'][0][0].shape

In [39]:
b = np.array(good_examples['orig']['v'][0]).shape

In [37]:
a == b

True

In [42]:
good_examples['neighbors']['v']

[array([[0.24234615, 0.10348432, 0.5118331 , ..., 0.36033654, 0.5667448 ,
         0.09381215],
        [0.2823501 , 0.11646202, 0.36502463, ..., 0.2891665 , 0.61796474,
         0.13190752],
        [0.34825417, 0.27901936, 0.33746684, ..., 0.60874546, 0.38208026,
         0.5029728 ],
        ...,
        [0.21959355, 0.15206262, 0.3589017 , ..., 0.57066035, 0.18339404,
         0.2891331 ],
        [0.19960175, 0.42059517, 0.62594783, ..., 0.3478138 , 0.15879217,
         0.28595105],
        [0.20172793, 0.05514861, 0.28671435, ..., 0.4509876 , 0.30387655,
         0.19386263]], dtype=float32),
 array([[0.48936182, 0.2954762 , 0.14425993, ..., 0.12827128, 0.39635703,
         0.49884117],
        [0.45803848, 0.22068338, 0.65286684, ..., 0.02713471, 0.3199094 ,
         0.16753413],
        [0.30960208, 0.06527094, 0.4083096 , ..., 0.06299937, 0.37265402,
         0.18082327],
        ...,
        [0.23027797, 0.2014927 , 0.34567437, ..., 0.00616849, 0.48087034,
         0.33480647

In [48]:
[qids_comp[q_id] for q_id in good_example_idxs]

[335653003, 302728002, 163933007, 358695004, 59079000]

In [54]:
for i, s in tqdm(enumerate(train_loader)):
    x = 1
    



0it [00:00, ?it/s]

3it [00:00, 23.73it/s]

7it [00:00, 28.31it/s]

11it [00:00, 29.88it/s]

15it [00:00, 30.77it/s]

19it [00:00, 31.17it/s]

23it [00:00, 31.63it/s]

27it [00:00, 31.93it/s]

31it [00:00, 32.19it/s]

35it [00:01, 32.34it/s]

39it [00:01, 32.25it/s]

43it [00:01, 32.25it/s]

47it [00:01, 32.22it/s]

51it [00:01, 32.23it/s]

55it [00:01, 32.24it/s]

59it [00:01, 32.18it/s]

63it [00:01, 32.21it/s]

67it [00:02, 32.17it/s]

71it [00:02, 32.21it/s]

75it [00:02, 32.19it/s]

79it [00:02, 32.19it/s]

83it [00:02, 32.19it/s]Process Process-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/datadrive/vqa.pytorch/vqa/lib/dataloader.py", line 32, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/datadrive/vqa.pytorch/vqa/lib/dataloader.py", line 32, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/datadrive/vqa.pytorch/vqa/datasets/vqa.py", line 72, in __getitem__
    item['answer'] = int(np.random.choice(item_vqa['answers_aid'], p=proba))
KeyboardInterrupt
